In [ ]:

import numpy as np
import pandas as pd


import os


In [ ]:
#clone YOLOv5 repo
%cd /kaggle/working/yolov5
!git clone https://github.com/ultralytics/yolov5

[Errno 2] No such file or directory: '/kaggle/working/yolov5'
/kaggle/working
Cloning into 'yolov5'...
remote: Enumerating objects: 17483, done.
remote: Total 17483 (delta 0), reused 0 (delta 0), pack-reused 17483 (from 1)
Receiving objects: 100% (17483/17483), 16.53 MiB | 26.41 MiB/s, done.
Resolving deltas: 100% (11988/11988), done.


In [ ]:
# installing requirments
%cd /kaggle/working/yolov5
%pip install -qr requirements.txt

/kaggle/working/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%cd /kaggle/working/yolov5
from yolov5 import utils
display=utils.notebook_init()

YOLOv5 🚀 v7.0-419-gcd44191c Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)


Setup complete ✅ (4 CPUs, 31.4 GB RAM, 6362.0/8062.4 GB disk)


In [ ]:
# import necessary libraries
import pandas as pd
import numpy as np
import torch
import cv2
from PIL import Image
import json
from ultralytics import YOLO


In [ ]:
test_path='/kaggle/input/26-class-object-detection-dataset/Senior-Design-VIAD-4/test'
train_path='/kaggle/input/26-class-object-detection-dataset/Senior-Design-VIAD-4/train'
valid_path='/kaggle/input/26-class-object-detection-dataset/Senior-Design-VIAD-4/valid'

In [ ]:
import os
os.mkdir('/kaggle/working/yolov5/data/coco')
os.mkdir('/kaggle/working/yolov5/data/coco/labels')
os.mkdir('/kaggle/working/yolov5/data/coco/images')

In [ ]:
def add_border(image,border_size,color):
    image_np=np.array(image)
    image_np=cv2.copyMakeBorder(image_np,border_size,border_size,border_size,border_size,
                               cv2.BORDER_CONSTANT,value=color)
    image=Image.fromarray(image_np)
    return image

In [ ]:
# extract labels data from json file
border_size=20
color=(0,0,0)

def create_labels(path_in,path_out):
    json_files = [file for file in os.listdir(path_in) if file.endswith('.json')]

    with open(os.path.join(path_in, json_files[0]), 'r') as f:
        data = json.load(f)
    df_images=pd.DataFrame(data['images'])
    df_images.rename(columns={'id':'image_id'},inplace=True)
    df_categories= pd.DataFrame(data['categories'])
    df_categories.rename(columns={'id':'category_id'},inplace=True)
    df_annotations=pd.DataFrame(data['annotations'])
    merged_df = pd.merge(df_annotations, df_categories, how='left',left_on='category_id', right_on='category_id')
    final_df = pd.merge(merged_df, df_images, how='left', left_on='image_id', right_on='image_id')
    df=final_df[['category_id','bbox','file_name',]]
    df[['bbox_x', 'bbox_y', 'bbox_width', 'bbox_height']] = pd.DataFrame(df['bbox'].apply(lambda x: pd.Series(x)))
    df.drop('bbox', axis=1, inplace=True)
    df.drop_duplicates(inplace=True)
    del df_images, df_categories, df_annotations, merged_df, final_df


    os.mkdir(f"/kaggle/working/yolov5/data/coco/labels/{path_out}")
    for file_name, group in df.groupby('file_name'):

        file_name = file_name[:-3] + "txt"
        label_filename = f"/kaggle/working/yolov5/data/coco/labels/{path_out}/{file_name}"

        with open(label_filename, 'w') as f:

            for index, row in group.iterrows():

                class_index = row['category_id']
                x_center = row['bbox_x'] + row['bbox_width'] / 2
                y_center = row['bbox_y'] + row['bbox_height'] / 2
                width = row['bbox_width']
                height = row['bbox_height']

                x_center = (x_center + 10) / 320
                y_center = (y_center + 10) / 320
                width /= 320
                height /= 320
                f.write(f"{class_index} {x_center} {y_center} {width} {height}\n")
    print('lable loaded')

    files = [file for file in os.listdir(path_in) if not file.endswith('.json')]
    os.mkdir(f"/kaggle/working/yolov5/data/coco/images/{path_out}")


    for file_name in files:
        source_file = os.path.join(path_in, file_name)
        destination_file = os.path.join(f'/kaggle/working/yolov5/data/coco/images/{path_out}', file_name)
        image = Image.open(source_file)
        image_with_border = add_border(image, border_size, color)
        image_with_border.save(destination_file)
    print('image loaded')

In [ ]:
create_labels(test_path,'test')
create_labels(train_path,'train')
create_labels(valid_path,'valid')

/tmp/ipykernel_35/1922852811.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['bbox_x', 'bbox_y', 'bbox_width', 'bbox_height']] = pd.DataFrame(df['bbox'].apply(lambda x: pd.Series(x)))
/tmp/ipykernel_35/1922852811.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['bbox_x', 'bbox_y', 'bbox_width', 'bbox_height']] = pd.DataFrame(df['bbox'].apply(lambda x: pd.Series(x)))
/tmp/ipykernel_35/1922852811.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

lable loaded
image loaded


/tmp/ipykernel_35/1922852811.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['bbox_x', 'bbox_y', 'bbox_width', 'bbox_height']] = pd.DataFrame(df['bbox'].apply(lambda x: pd.Series(x)))
/tmp/ipykernel_35/1922852811.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['bbox_x', 'bbox_y', 'bbox_width', 'bbox_height']] = pd.DataFrame(df['bbox'].apply(lambda x: pd.Series(x)))
/tmp/ipykernel_35/1922852811.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

lable loaded
image loaded


/tmp/ipykernel_35/1922852811.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['bbox_x', 'bbox_y', 'bbox_width', 'bbox_height']] = pd.DataFrame(df['bbox'].apply(lambda x: pd.Series(x)))
/tmp/ipykernel_35/1922852811.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['bbox_x', 'bbox_y', 'bbox_width', 'bbox_height']] = pd.DataFrame(df['bbox'].apply(lambda x: pd.Series(x)))
/tmp/ipykernel_35/1922852811.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

lable loaded
image loaded


In [ ]:
model=YOLO('yolov8m.pt')

100%|██████████| 49.7M/49.7M [00:00<00:00, 236MB/s]


In [ ]:
# create yaml file to provide path of data
yaml_content = """
train: /kaggle/working/yolov5/data/coco/images/train
val: /kaggle/working/yolov5/data/coco/images/valid
nc: 37
names: ['cars-bikes-people', 'Bus', 'Bushes', 'Person', 'Truck', 'backpack', 'bench',
             'bicycle', 'boat', 'branch', 'car', 'chair', 'clock', 'crosswalk', 'door',
             'elevator', 'fire_hydrant', 'green_light', 'gun', 'handbag', 'motorcycle',
             'person', 'pothole', 'rat', 'red_light', 'scooter', 'sheep', 'stairs', 'stop_sign',
             'suitcase', 'traffic light', 'traffic_cone', 'train', 'tree', 'truck', 'umbrella',
              'yellow_light']
"""

with open("data.yaml", "w") as f:
    f.write(yaml_content)

In [ ]:
#train model on given epoch
model.train(data='/kaggle/working/yolov5/data.yaml',epochs=3)

Ultralytics 8.3.146 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/yolov5/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=3, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretra

train: Scanning /kaggle/working/yolov5/data/coco/labels/train.cache... 31944 images, 567 backgrounds, 0 corrupt: 100%|██████████| 32511/32511 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 321.4±204.1 MB/s, size: 16.3 KB)


val: Scanning /kaggle/working/yolov5/data/coco/labels/valid.cache... 4189 images, 57 backgrounds, 0 corrupt: 100%|██████████| 4246/4246 [00:00<?, ?it/s]


Plotting labels to runs/detect/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000244, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train3
Starting training for 3 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       4246       7242      0.665      0.447      0.479      0.308



3 epochs completed in 1.085 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 52.1MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 52.1MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics 8.3.146 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,861,183 parameters, 0 gradients, 78.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 133/133 [01:08<00:00,  1.95it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all       4246       7242      0.665      0.447      0.479      0.308
                   Bus        203        275      0.733      0.804      0.807      0.632
                Bushes        207        437      0.884      0.644      0.665      0.414
                Person        191       1023      0.532      0.107      0.189     0.0698
                 Truck         37         45      0.637      0.356      0.429      0.338
              backpack          3          3          1          0          0          0
                 bench          2          3          1          0          0          0
               bicycle        290        358      0.633      0.713      0.718      0.444
                  boat          2          4          0          0          0          0
                branch         21         62       0.45      0.242      0.262      0.125
                   car        228        463      0.816      0.544      0.668       0.45
                 chai

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d7a81466b90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038, 

In [ ]:
infer=YOLO('/kaggle/working/yolov5/runs/detect/train3/weights/best.pt')

In [ ]:
infer.predict('/kaggle/working/yolov5/data/coco/images/test',save=True,save_txt=True)


WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/2165 /kaggle/working/yolov5/data/coco/images/test/-0573F94C-289D-4503-A571-94C6C8C5674C-png_jpg.rf.203ce892cfb5f1007edfffc3583c559c.jpg: 640x640 1 car, 37.2ms
image 2/2165 /kaggle/working/yolov5/data/coco/images/test/-13AD8372-CDDB-4DCF-B8F9-DC10D210064B-png_jpg.rf.e8217e5555bb565dc324989173877f5a.jpg: 640x640 1 car, 35.5ms
image 3/2165 /kaggle/working/yolov5/data/coco/images/test/-1F999A49-2799-4FBC-811D-D10A165F8B02-png_jpg.rf.8fc376c8a650e3c

# Thank You